In [1]:
import json
import pandas as pd

from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# from transformers import ElectraForPreTraining, ElectraTokenizerFast
from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification, pipeline
from transformers import AdamW

In [6]:
from transformers import AutoTokenizer
checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\Admin\anaconda3\envs\SupportSystem\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split_part_dialog(
    df['dialog'], df['initial_emotion_intensity'],
    start_percentage=0, end_percentage=0.2,
    test_start=0.2, test_end=0.4)
# standard division on test and train
train_dataset, val_dataset, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2,
                                                                        random_state=1)

In [7]:
inputs = tokenizer(df['dialog'], padding=True, truncation=True, return_tensors=True)
print(inputs)

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [2]:
from transformers import pipeline

classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

sentences = ["I am not having a great day"]

model_outputs = classifier(sentences)
print(model_outputs[0])
# produces a list of dicts for each of the labels


[{'label': 'disappointment', 'score': 0.46669498085975647}, {'label': 'sadness', 'score': 0.39849525690078735}, {'label': 'annoyance', 'score': 0.06806591153144836}, {'label': 'neutral', 'score': 0.057030171155929565}, {'label': 'disapproval', 'score': 0.044239308685064316}, {'label': 'nervousness', 'score': 0.014850764535367489}, {'label': 'realization', 'score': 0.014059898443520069}, {'label': 'approval', 'score': 0.0112674655392766}, {'label': 'joy', 'score': 0.00630340026691556}, {'label': 'remorse', 'score': 0.006221492309123278}, {'label': 'caring', 'score': 0.006029403302818537}, {'label': 'embarrassment', 'score': 0.005265483167022467}, {'label': 'anger', 'score': 0.004981426056474447}, {'label': 'disgust', 'score': 0.004259031731635332}, {'label': 'grief', 'score': 0.0040021371096372604}, {'label': 'confusion', 'score': 0.0033829212188720703}, {'label': 'relief', 'score': 0.003140498884022236}, {'label': 'desire', 'score': 0.00282747158780694}, {'label': 'admiration', 'score'

In [ ]:
from transformers import pipeline
model="siebert/sentiment-roberta-large-english"
sentiment_analysis = pipeline("sentiment-analysis",model=model)
#print(sentiment_analysis("I love this!"))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Define hyperparameters
num_epochs = 5
batch_size = 5
learning_rate = 2e-5

In [ ]:
# tokenizer = RobertaTokenizerFast.from_pretrained("arpanghoshal/EmoRoBERTa")
# model = TFRobertaForSequenceClassification.from_pretrained("arpanghoshal/EmoRoBERTa")
# 
# emotion = pipeline('sentiment-analysis', model='arpanghoshal/EmoRoBERTa')
# emotion.to(device)

In [ ]:
# Load tokenizer and model
# model = ElectraForPreTraining.from_pretrained("google/electra-base-discriminator")
# tokenizer = ElectraTokenizerFast.from_pretrained("google/electra-base-discriminator")
# num_classes = 5
# #model = ElectraForSequenceClassification.from_pretrained('google/bio-electra-base-discriminator', num_labels=num_classes)
# model.to(device)

# Prepare data

In [ ]:
def load_data(file_path):
    with open(file_path, "r", encoding='utf-8') as file:
        data = json.load(file)
    return data

dataset = load_data("ESConv.json")
dataframe = pd.DataFrame(dataset)

In [ ]:
def extract_dialog(dialog, start_percentage, end_percentage):
    if isinstance(dialog, list):
        seeker_contents = [item['content'] for item in dialog if item['speaker'] == 'seeker']
        start_index = int(start_percentage * len(seeker_contents))
        end_index = int(end_percentage * len(seeker_contents))
        return ' '.join(seeker_contents[start_index:end_index])
    elif isinstance(dialog, str):
        sentences = dialog.split('.')
        seeker_contents = [sentence for sentence in sentences] #[str, str, ..., str]
        start_index = int(start_percentage * len(seeker_contents))
        end_index = int(end_percentage * len(seeker_contents))
        return seeker_contents[start_index:end_index]
    else:
        return None

In [ ]:
df = pd.DataFrame()
df['dialog'] = dataframe['dialog'].apply(lambda x: extract_dialog(x, 0, 1)) #take whole dialog from seeker
df['dialog'] = df['dialog'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

df['initial_emotion_intensity'] = dataframe['survey_score'].apply(
    lambda x: x['seeker']['initial_emotion_intensity'])
df['initial_emotion_intensity'].dropna(inplace=True)
df['initial_emotion_intensity'] = df['initial_emotion_intensity'].astype(int)
df['dialog'].dropna(inplace=True)
#train_loader = DataLoader(df, batch_size=32, shuffle=True)

In [ ]:
df

In [ ]:
def train_test_split_part_dialog(dialog, labels, start_percentage, end_percentage, test_start, test_end):
    train_data = []
    test_data = []

    if isinstance(labels, pd.Series):
        labels = labels.tolist()

    # Ensure labels are within the range of 0 to n_classes - 1
    min_label = min(labels)
    train_labels = labels
    test_labels = labels

    if isinstance(dialog, list):
        for conv in dialog:
            seeker_contents = [item['content'] for item in conv if item['speaker'] == 'seeker']
            start_index = int(start_percentage * len(seeker_contents))
            end_index = int(end_percentage * len(seeker_contents))
            test_start_index = int(test_start * len(seeker_contents))
            test_end_index = int(test_end * len(seeker_contents))

            train_data.extend(seeker_contents[start_index:end_index])
            test_data.extend(seeker_contents[test_start_index:test_end_index])

    elif isinstance(dialog, str):
        sentences = dialog.split('.')
        seeker_contents = [sentence.strip() for sentence in sentences]
        start_index = int(start_percentage * len(seeker_contents))
        end_index = int(end_percentage * len(seeker_contents))
        test_start_index = int(test_start * len(seeker_contents))
        test_end_index = int(test_end * len(seeker_contents))

        train_data = seeker_contents[start_index:end_index]
        test_data = seeker_contents[test_start_index:test_end_index]

    elif isinstance(dialog, pd.Series):
        for conv in dialog:
            seeker_contents = conv
            start_index = int(start_percentage * len(seeker_contents))
            end_index = int(end_percentage * len(seeker_contents))
            test_start_index = int(test_start * len(seeker_contents))
            test_end_index = int(test_end * len(seeker_contents))

            # train_data.append((seeker_contents[start_index:end_index],))  # Append as tuple
            # test_data.append((seeker_contents[test_start_index:test_end_index],))  # Append as tuple
            train_data.append(' '.join(seeker_contents[start_index:end_index]))
            test_data.append(' '.join(seeker_contents[test_start_index:test_end_index]))

    return tuple(train_data), tuple(test_data), train_labels, test_labels

In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split_part_dialog(
    df['dialog'], df['initial_emotion_intensity'],
    start_percentage=0, end_percentage=0.2,
    test_start=0.2, test_end=0.4)

In [ ]:
# standard division on test and train
train_dataset, val_dataset, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=1)

In [ ]:
for i in train_dataset:
    print(i)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
for i, batch in train_dataloader:
    print("Texts:", batch)
    print("Labels:", i)

In [ ]:
# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=learning_rate, no_deprecation_warning=True)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0

    for i, batch in train_dataloader:
        print('Training...')
        inputs = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')
        inputs = {k: v.to(device) for k, v in inputs.items()}
        labels = i.to(device)

        optimizer.zero_grad()
        outputs = model(**inputs)
        logits = outputs.logits
        loss = loss_fn(logits, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted_labels = torch.max(logits, dim=1)
        total_correct += (predicted_labels == labels).sum().item()
        total_samples += labels.size(0)
        print('Stop Training...')

    train_loss = total_loss / len(train_dataloader)
    train_accuracy = total_correct / total_samples

    # Evaluation on validation set
    model.eval()
    with torch.no_grad():
        total_val_loss = 0
        total_val_correct = 0
        total_val_samples = 0

        for i, batch in val_dataloader:
            print('Validation...')
            inputs = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')
            inputs = {k: v.to(device) for k, v in inputs.items()}
            labels = i.to(device)

            outputs = model(**inputs)
            logits = outputs.logits
            loss = loss_fn(logits, labels)

            total_val_loss += loss.item()
            _, predicted_labels = torch.max(logits, dim=1)
            total_val_correct += (predicted_labels == labels).sum().item()
            total_val_samples += labels.size(0)

        val_loss = total_val_loss / len(val_dataloader)
        val_accuracy = total_val_correct / total_val_samples

    # Print epoch results
    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"Train Loss: {train_loss:.4f} - Train Accuracy: {train_accuracy:.4f}")
    print(f"Validation Loss: {val_loss:.4f} - Validation Accuracy: {val_accuracy:.4f}")